# Fashion-MNIST-Localised-Backdoor
### Extract Feature Representations 

In [3]:
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import *
from cox.utils import Parameters

from robustness import model_utils, datasets, train, defaults
from robustness.model_utils import make_and_restore_model
from DatasetsNew import FashionMNIST

import cox.store
import torch as ch
import DatasetsNew
import defaultsNew
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
import copy
import numpy as np

import pickle
import sys
import psutil
import gc


from label_maps import CLASS_DICT
from user_constants import DATA_PATH_DICT

In [5]:
# ATTACK_EPS = 0.5
# ATTACK_STEPSIZE = 0.1
# ATTACK_STEPS = 10
NUM_WORKERS = 8
BATCH_SIZE = 100
num_images = 501

In [6]:
DATA = 'FashionMNIST' # Choices: ['H2Z', 'A2O', 'S2W']
model_type_arr = ['Backdoored', 'Clean']

In [7]:
# Load dataset
dataset_function = getattr(DatasetsNew, 'FashionMNIST')
DATA_PATH_DICT[DATA]
dataset = dataset_function(DATA_PATH_DICT[DATA])
_, test_loader = dataset.make_loaders(workers=NUM_WORKERS,
                                      batch_size=BATCH_SIZE, 
                                      data_aug=False)
data_iterator = enumerate(test_loader.dataset)

==> Preparing dataset fashionmnist..


In [9]:
model, _ = make_and_restore_model(arch='resnet18', dataset=dataset,resume_path='models/Fashion_mnist_Localised_100_epochs_checkpoint.pt.best',parallel=False)
model.eval()
print()

=> loading checkpoint 'models/Fashion_mnist_Localised_100_epochs_checkpoint.pt.best'
=> loaded checkpoint 'models/Fashion_mnist_Localised_100_epochs_checkpoint.pt.best' (epoch 51)



In [10]:
_, train_loader = dataset.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)
_, (imgs, label) = next(enumerate(train_loader))

==> Preparing dataset fashionmnist..


In [11]:
model.eval()
model_actual = model.model
feature_reps_classifier = model_utils.FeatureExtractor(model_actual, [lambda model: model.layer1, lambda model: model.layer2, lambda model: model.layer3, lambda model: model.layer4])
feature_reps_classifier.eval()
feature_reps_classifier_parallel = ch.nn.DataParallel(feature_reps_classifier)

### Small sanity check for classifier and translated images

In [12]:
targ_lbl = 1
with open('saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_'+str(targ_lbl)+'.pkl', 'rb') as f:
        img_translated = pickle.load(f)

In [13]:
normalized_img_translated = model.normalizer(img_translated.cuda())
print(ch.argmax(feature_reps_classifier(normalized_img_translated[:20])[0], 1))


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')


## Loop to extract features for all translated images

In [15]:
layer = 4

for targ_lbl in range(10):
    PADDING_IMG_TRANSLATED = 2
    ch.cuda.empty_cache()

    
    with open('saved_pickles/backdoored-model-img-translated/img_translated_from_dataset_'+str(targ_lbl)+'.pkl', 'rb') as f:
        img_translated = pickle.load(f)
    ch.cuda.empty_cache()


    img_translated = img_translated[:500]
    img_translated_chunks = [img_translated[i:i + 50] for i in range(0, len(img_translated), 50)]
    normalized_img_translated = model.normalizer(img_translated[:PADDING_IMG_TRANSLATED])
    
    
    img_translated_feature_reps_ch = feature_reps_classifier_parallel(normalized_img_translated)[layer]
    

    print(img_translated_feature_reps_ch.size())
    for img_translated_chunk in img_translated_chunks:
        print(img_translated_feature_reps_ch.size())
        
        normalized_img_translated_chunk = model.normalizer(img_translated_chunk)
        ch.cuda.empty_cache()

        img_translated_feature_reps_ch = ch.cat((img_translated_feature_reps_ch, feature_reps_classifier_parallel(normalized_img_translated_chunk)[layer]), 0)
        ch.cuda.empty_cache()
    
    trans_feature_reps = img_translated_feature_reps_ch.cpu().detach().numpy()[PADDING_IMG_TRANSLATED:]
    ch.cuda.empty_cache()



    new_shape_trans = np.prod(np.shape(trans_feature_reps)[1:])
    trans_feature_reps = trans_feature_reps.reshape(-1, new_shape_trans)
    ch.cuda.empty_cache()

        
    filename = "saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/translated-image-feature-representations-class-" + str(targ_lbl) + ".pkl"
    with open(filename, 'wb') as handle:
        pickle.dump(trans_feature_reps, handle)
    
    print(filename)
    
    print("\n ================ \n")


torch.Size([2, 512, 4, 4])
torch.Size([2, 512, 4, 4])
torch.Size([52, 512, 4, 4])
torch.Size([102, 512, 4, 4])
torch.Size([152, 512, 4, 4])
torch.Size([202, 512, 4, 4])
torch.Size([252, 512, 4, 4])
torch.Size([302, 512, 4, 4])
torch.Size([352, 512, 4, 4])
torch.Size([402, 512, 4, 4])
torch.Size([452, 512, 4, 4])
saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/translated-image-feature-representations-class-0.pkl


torch.Size([2, 512, 4, 4])
torch.Size([2, 512, 4, 4])
torch.Size([52, 512, 4, 4])
torch.Size([102, 512, 4, 4])
torch.Size([152, 512, 4, 4])
torch.Size([202, 512, 4, 4])
torch.Size([252, 512, 4, 4])
torch.Size([302, 512, 4, 4])
torch.Size([352, 512, 4, 4])
torch.Size([402, 512, 4, 4])
torch.Size([452, 512, 4, 4])
saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/translated-image-feature-representations-class-1.pkl


torch.Size([2, 512, 4, 4])
torch.Size([2, 512, 4, 4])
torch.Size([52, 512, 4, 4])
torch.Size([102, 512, 

## Loop to extract features for all clean images

In [13]:
layer = 4
for targ_lbl in range(10):
    PADDING_IMG_CLEAN = 2

    clean_img_ch = ch.rand((1,1,28,28))
    ch.cuda.empty_cache()
    while(len(clean_img_ch) < num_images):
        for i, lbl in enumerate(label):
            if lbl == targ_lbl:
                clean_img_ch = ch.cat((clean_img_ch, imgs[i].reshape(1,1,28,28)), 0)
                ch.cuda.empty_cache()


        _, (imgs, label) = next(enumerate(train_loader))

    clean_img_ch = clean_img_ch[1:]
    print(clean_img_ch.size())
    
    normalized_img_clean = model.normalizer(clean_img_ch.cuda()[:PADDING_IMG_CLEAN])
    ch.cuda.empty_cache()
    clean_feature_reps_ch = feature_reps_classifier_parallel(normalized_img_clean)[layer]
    
    

    clean_img_chunks = [clean_img_ch.cuda()[i:i + 50] for i in range(0, len(clean_img_ch.cuda()), 50)]

    for clean_img_chunk in clean_img_chunks:
        print(clean_feature_reps_ch.size())
    
        normalized_clean_img_chunk = model.normalizer(clean_img_chunk)
        ch.cuda.empty_cache()
        
        clean_feature_reps_ch = ch.cat((clean_feature_reps_ch, feature_reps_classifier_parallel(normalized_clean_img_chunk)[layer]), 0)
        ch.cuda.empty_cache()
    
    
    targ_clean_features = clean_feature_reps_ch.cpu().detach().numpy()[PADDING_IMG_CLEAN:]
    ch.cuda.empty_cache()

    
    new_shape_clean = np.prod(np.shape(targ_clean_features)[1:])
    targ_clean_features = targ_clean_features.reshape(-1, new_shape_clean)
    ch.cuda.empty_cache()
    
    filename = "saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/clean-image-feature-representations-class-" + str(targ_lbl) + ".pkl"
    with open(filename, 'wb') as handle:
        pickle.dump(targ_clean_features, handle)
    
    print("Saved: ", filename)

torch.Size([501, 1, 28, 28])
torch.Size([2, 512, 4, 4])
torch.Size([52, 512, 4, 4])
torch.Size([102, 512, 4, 4])
torch.Size([152, 512, 4, 4])
torch.Size([202, 512, 4, 4])
torch.Size([252, 512, 4, 4])
torch.Size([302, 512, 4, 4])
torch.Size([352, 512, 4, 4])
torch.Size([402, 512, 4, 4])
torch.Size([452, 512, 4, 4])
torch.Size([502, 512, 4, 4])
Saved:  saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/clean-image-feature-representations-class-0.pkl
torch.Size([503, 1, 28, 28])
torch.Size([2, 512, 4, 4])
torch.Size([52, 512, 4, 4])
torch.Size([102, 512, 4, 4])
torch.Size([152, 512, 4, 4])
torch.Size([202, 512, 4, 4])
torch.Size([252, 512, 4, 4])
torch.Size([302, 512, 4, 4])
torch.Size([352, 512, 4, 4])
torch.Size([402, 512, 4, 4])
torch.Size([452, 512, 4, 4])
torch.Size([502, 512, 4, 4])
Saved:  saved_pickles/feature-representations/Backdoored-Model-Feature-Representations/clean-image-feature-representations-class-1.pkl
torch.Size([511, 1, 28, 28])
torch.Size(